# Week 3: Sequence models (RNN, LSTM)
Tokenizing words gave us a negative effect last week because the context of words was hard to follow when the words were broken down into subwords.
The sequence in which the tokens of subwords appear becomes very important when understanding their meaning.

In RNN, apart from x and y, there is always an element that is fed into a function from a previous function.
```
y0    y1    y2
^     ^     ^
| f0  | f1  | f2
F --> F --> F -->
^     ^     ^
|     |     |
x0    x1    x2
```

There can be a limitation when approaching text classification in this way:
`Today has a beautiful blue <...>` -> "blue sky" - the context word that lets us understand the next word is very close to it.

`I lived in Ireland, so at school they made me learn how to speak <...>` -> "to speak Galeic" - word "Ireland" is a context word but it apeears very far from the predicting word.

In this case using LSTM (Long Short-Term Memory) network can be useful to use. LSTM has an additional pipeline of context called "Cell state" and this can be passed through the network.
This helps to keep context from earlier tokens relevant to later ones.
Cell State can also be bi-directional: later contexts can impact earlier ones.